In [1]:
import yaml
import sys
sys.path.append("../")
from plugins.interfaces.diagnostics import AWAFrameGrabberDiagnostic, ROI

# define screen and trim parameters
screen = "DYG5"
magnet_info = yaml.safe_load(open("awa_config/awa_drive_magnet_config.yml"))
screen_info = yaml.safe_load(open("awa_config/awa_drive_camera_config.yml"))[screen]

In [2]:
# create Xopt objects
from xopt import VOCS
IMAGE_CONSTRAINTS = {
            "bb_penalty": ["LESS_THAN", 0.0],
        }

vocs = VOCS(
    variables = {
        "AWA:Bira3Ctrl:Ch05":[-1.75,1.75], # DQ6
        "AWA:Drive:DS3:Ctrl": [130.0,250.0], # Matching solenoid
        "AWA:Drive:DS1:Ctrl": [450.0,550.0], # Bucking solenoid
    },
    constraints = IMAGE_CONSTRAINTS,
    observables = ["Sx_squared", "Sy_squared"],
)

In [3]:
vocs.variable_names

['AWA:Bira3Ctrl:Ch05', 'AWA:Drive:DS1:Ctrl', 'AWA:Drive:DS3:Ctrl']

In [4]:
screen_roi = ROI(
    xcenter=int(screen_info["center"][0]),
    ycenter=int(screen_info["center"][1]),
    xwidth=int(screen_info["radius"]*1.5),
    ywidth=int(screen_info["radius"]*1.5),
)

image_diagnostic = AWAFrameGrabberDiagnostic(
    roi=screen_roi, apply_bounding_box_constraint=True, visualize=False,
    save_image_location="../../awa_data/03_14_BAX/"
)

In [5]:
image_diagnostic

AWAFrameGrabberDiagnostic(screen_name='AWANIFrameGrabber', ip_address='N/A', alias=None, array_data_suffix='N/A', array_n_cols_suffix='N/A', array_n_rows_suffix='N/A', resolution_suffix=None, resolution=1.0, beam_shutter_pv=None, extra_pvs=[], background_file=None, save_image_location='../../awa_data/03_14_BAX/', roi=ROI(xcenter=293, ycenter=248, xwidth=310, ywidth=310), gain=1.0, min_log_intensity=4.0, bounding_box_half_width=3.0, wait_time=1.0, n_fitting_restarts=1, visualize=False, verbose=True, return_statistics=False, threshold=0.0, apply_bounding_box_constraint=True, target_charge=None, target_charge_pv=None, charge_atol=0.1, testing=False)

In [6]:
import time
from xopt import Evaluator
from epics import caput

def evaluate(inputs: dict):
    global image_diagnostic
    # caput values
    for name, val in inputs.items():
        caput(name, val)

    # wait for changes to occur - use small wait time for interpolated measurements
    time.sleep(2.0)

    results = image_diagnostic.measure_beamsize(1, **inputs)
    results["Sx_m"] = results["Sx"] * 25 / screen_info["radius"]
    results["Sx_squared"] = results["Sx_m"]**2

    results["Sy_m"] = results["Sy"] * 25 / screen_info["radius"]
    results["Sy_squared"] = results["Sy_m"]**2
    
    # measure distance to image center
    print(results["Cx"], image_diagnostic.roi.xwidth/2)
    results["center_dist_x"] = results["Cx"] - image_diagnostic.roi.xwidth/2
    results["center_dist_y"] = results["Cy"] - image_diagnostic.roi.ywidth/2

    results["center_dist"] = (results["center_dist_x"]**2 + results["center_dist_y"]**2)**0.5
    results["time"] = time.time()
    
    return results

evaluator = Evaluator(function=evaluate)

In [7]:
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from gpytorch.priors.torch_priors import GammaPrior
from xopt.generators.bayesian.bax_generator import BaxGenerator
from xopt.generators.bayesian.bayesian_exploration import BayesianExplorationGenerator
from copy import deepcopy
sys.path.append("../../emitopt/")
import torch
from emitopt.algorithms import ScipyMinimizeEmittanceXY

meas_dim = 0
tuning_dims = [1,2]
covar_module = (MaternKernel(ard_num_dims=len(tuning_dims), 
                              active_dims=tuning_dims, 
                              lengthscale_prior=None) * 
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )
scaled_covar_module = ScaleKernel(covar_module)   
    
# prepare options for Xopt generator
covar_module_dict = {'Sx_squared': scaled_covar_module, "Sy_squared": deepcopy(scaled_covar_module)}
# covar_module_dict = {}
model_constructor = StandardModelConstructor(
    covar_modules=covar_module_dict, use_low_noise_prior=True
)

from emitopt.utils import get_quad_scale_factor
q_len=0.1

pv_scale = (q_len*10 / 0.893) # [kG] / [A] for AWA Blue quads
geo_scale_factor = get_quad_scale_factor(E=0.060, q_len=q_len) # [m^-2] / [kG] 


scale_factor = pv_scale * geo_scale_factor
rmat_x = torch.tensor(((1.0,2.375),(0.0,1.0)))
rmat_y = torch.tensor(((1.0,2.375),(0.0,1.0)))
n_samples=10

algo_kwargs = {
        'x_key': 'Sx_squared',
        'y_key': 'Sy_squared',
        'scale_factor': scale_factor,
        'q_len': q_len,
        'rmat_x': rmat_x,
        'rmat_y': rmat_y,
        'n_samples': n_samples,
        'meas_dim': meas_dim,
        'n_steps_measurement_param': 11,
#         'scipy_options': None,
        'thick_quad': False,
}
algo = ScipyMinimizeEmittanceXY(**algo_kwargs)

generator = BaxGenerator(
    vocs=vocs, 
    gp_constructor=model_constructor, 
    algorithm=algo,
    #n_interpolate_points=5,
)

#generator = BayesianExplorationGenerator(
#    vocs=vocs, 
#    gp_constructor=model_constructor, 
#)

generator.numerical_optimizer.max_time = 10.0


In [8]:
from xopt import Xopt
X = Xopt(vocs=vocs, generator=generator, 
         evaluator=evaluator, strict=True,dump_file="bax_emittance_3d_x_y.yml")

In [9]:
X


            Xopt
________________________________
Version: 2.2.1+8.g7a7eff90.dirty
Data size: 0
Config as YAML:
dump_file: bax_emittance_3d_x_y.yml
evaluator:
  function: __main__.evaluate
  function_kwargs: {}
  max_workers: 1
  vectorized: false
generator:
  algorithm:
    n_samples: 10
  algorithm_results_file: null
  computation_time: null
  fixed_features: null
  gp_constructor:
    covar_modules: {}
    mean_modules: {}
    name: standard
    trainable_mean_keys: []
    transform_inputs: true
    use_low_noise_prior: true
  log_transform_acquisition_function: false
  max_travel_distances: null
  model: null
  n_candidates: 1
  n_interpolate_points: null
  n_monte_carlo_samples: 128
  name: BAX
  numerical_optimizer:
    max_iter: 2000
    max_time: 10.0
    n_restarts: 20
    name: LBFGS
  turbo_controller: null
  use_cuda: false
max_evaluations: null
serialize_inline: false
serialize_torch: false
strict: true
vocs:
  constants: {}
  constraints:
    bb_penalty:
    - LESS_THAN


In [10]:
from epics import caget_many
from xopt.utils import get_local_region
import pandas as pd
import numpy as np
current_value = dict(zip(X.vocs.variable_names, caget_many(X.vocs.variable_names)))
print(current_value)

# create a mesh
n_samples = 3
xx = np.meshgrid(
    np.linspace(-0.5, 0.5,n_samples),
    np.linspace(130.0, 250.0,n_samples),
    np.linspace(500.0, 550.0,n_samples)

)
pts = np.vstack([ele.flatten() for ele in xx])
print(len(pts))

import yaml
#with open("bax_scan_smaller_range.yml") as stream:
#    d =yaml.safe_load(stream)
#X.add_data(pd.DataFrame(d['data']))


X.evaluate_data({
    "AWA:Bira3Ctrl:Ch05":pts[0],
    "AWA:Drive:DS3:Ctrl":pts[1],
    "AWA:Drive:DS1:Ctrl":pts[2],

});
#import yaml
#X.random_evaluate(2)#, custom_bounds=random_sample_region)

{'AWA:Bira3Ctrl:Ch05': 0.79622, 'AWA:Drive:DS1:Ctrl': 550.0, 'AWA:Drive:DS3:Ctrl': 141.70598638723578}
3
fitting image


CA.Client.Exception...............................................
    Context: "Channel: "AWANIFG:ImgData", Connecting to: 192.168.2.57:62521, Ignored: 192.168.0.2:62521"
    Source File: ../cac.cpp line 1320
    Current Time: Fri Mar 15 2024 16:53:12.206118528
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "AWANIFG:ImgData", Connecting to: 192.168.2.57:62521, Ignored: awa3.hep.anl.gov:62521"
    Source File: ../cac.cpp line 1320
    Current Time: Fri Mar 15 2024 16:53:12.207083101
..................................................................


154.41267472110175 155.0
fitting image
152.93948966762315 155.0
fitting image
153.18401719521594 155.0
fitting image
139.675429718798 155.0
fitting image
135.96108913259798 155.0
fitting image
132.78168986295245 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
115.84708667649255 155.0
fitting image
158.2472229158813 155.0
fitting image
158.95231377906225 155.0
fitting image
160.54136082688956 155.0
fitting image
144.5775053158504 155.0
fitting image
145.1490089578104 155.0
fitting image
150.24493135289424 155.0
fitting image
127.23339248632932 155.0
fitting image
133.46121771870733 155.0
fitting image
132.9873803669633 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0
fitting image
nan 155.0


In [12]:
for i in range(40):
    X.step()

/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)
/home/awa/awa-badger/control_notebooks/../../emitopt/emitopt/sampling.py:117: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1691.)
  Lnn = torc

fitting image
195.23285368599127 155.0


/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)


fitting image
128.9027919083571 155.0


/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)


...

/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)


fitting image
100.30939673610034 155.0


/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)


fitting image
172.1807627437067 155.0


CA.Client.Exception...............................................
    Context: "192.168.0.166:5064"
    Source File: ../tcpiiu.cpp line 926
    Current Time: Fri Mar 15 2024 17:14:59.520054805
..................................................................
Unexpected problem with CA circuit to server "192.168.0.166:5064" was "No route to host" - disconnecting
CA.Client.Exception...............................................
    Context: "192.168.0.166:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Fri Mar 15 2024 17:30:25.956280034
..................................................................


In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
X.generator.train_model()
fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch05","AWA:Drive:DS3:Ctrl"],
    output_names=["Sx_squared"],
)

fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch05","AWA:Drive:DS1:Ctrl"],
    output_names=["Sx_squared"],
)

#X_stars = X.generator.algorithm_results["X_stars"]
#for ele in X_stars:
#    ax[0,0].axhline(ele,c="C3", xmax=0.1)


In [ ]:
fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch05"],
    output_names=["Sx_squared"],
    reference_point={"AWA:Drive:DS3:Ctrl":180.0,"AWA:Drive:DS1:Ctrl":525.0},
)


In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
X.generator.train_model()
fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch05","AWA:Drive:DS3:Ctrl"],
    output_names=["Sx_squared"],
)

In [ ]:
from emitopt.plot_utils import plot_virtual_emittance_vs_tuning,plot_pathwise_emittance_vs_tuning
X.generator.algorithm.thick_quad = False
plot_virtual_emittance_vs_tuning(X, torch.tensor([[525.,170.0]]),n_points=100)

In [ ]:
from emitopt.plot_utils import plot_virtual_measurement_scan
plot_virtual_measurement_scan(X, torch.tensor([[220.]]))

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model

fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch03"],
    reference_point={"AWA:Drive:DT8H_B_S:Ctrl":0.25},
    output_names=["Cx"]
)

fig2,ax2 = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch03"],
    reference_point={"AWA:Drive:DT8H_B_S:Ctrl":1.0},
    output_names=["Cx"]
)

fig3,ax3 = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch03"],
    reference_point={"AWA:Drive:DT8H_B_S:Ctrl":1.1},
    output_names=["Cx"]
)

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
X.generator.train_model()
fig,ax = visualize_generator_model(
    X.generator, 
    variable_names=["AWA:Bira3Ctrl:Ch03","AWA:Drive:DT8H_B_S:Ctrl"],
    output_names=["Cx"]
)
#X_stars = X.generator.algorithm_results["X_stars"]
#for ele in X_stars:
#    ax[0,0].axhline(ele,c="C3", xmax=0.1)


In [ ]:
import yaml
with open("bax_scan_1.yml") as stream:
    d =yaml.safe_load(stream)
import pandas as pd
import torch
df =pd.DataFrame(d['data'])
df

df2 =df[df['AWA:Drive:DS3:Ctrl'] == 230.]
df2
df3 =df2.dropna()
df3

x = torch.tensor(df3['AWA:Bira3Ctrl:Ch03'])
y = torch.tensor(df3['Sx_squared'])

from emitopt.utils import get_quad_scale_factor
q_len = 0.1
pv_scale = (q_len*10 / 0.893) # [kG] / [A] for AWA Blue quads
geo_scale_factor = get_quad_scale_factor(E=0.060, q_len=q_len) # [m^-=2] / [kG]

scale_factor =pv_scale * geo_scale_factor
k =scale_factor * x
k

import matplotlib.pyplot as plt
plt.scatter(k, y)
plt.figure()

from emitopt.analysis import compute_emit_bmag, compute_emit_bayesian
rmat_x =torch.tensor(((1.0,2.375),(0.0,1.0)))
#emit = compute_emit_bmag(k, y, q_len, rmat_x)[0]
emit =compute_emit_bayesian(
    k, y.sqrt(), q_len, 
    rmat_x,visualize=True,noise_prior=GammaPrior(1.0, 10.0))[0]
plt.hist(emit)


In [ ]:
x_tuning =  torch.tensor([[220.]])
n_samples = 100

bax_model_ids = [
    X.generator.vocs.output_names.index(name)
    for name in X.generator.algorithm.observable_names_ordered
]
bax_model = X.generator.model.subset_output(bax_model_ids)

x = X.generator.algorithm.get_meas_scan_inputs(x_tuning, X.generator.vocs.bounds) # result shape n_tuning_configs*n_steps x ndim

p = bax_model.posterior(x)
bss = p.sample(torch.Size([n_samples])) # result shape n_samples x n_tuning_configs*n_steps x num_outputs (1 or 2)

x = x.reshape(x_tuning.shape[0], X.generator.algorithm.n_steps_measurement_param, -1) # result n_tuning_configs x n_steps x ndim
x = x.repeat(n_samples,1,1,1)
# result shape n_samples x n_tuning_configs x n_steps x ndim
bss = bss.reshape(n_samples, x_tuning.shape[0], X.generator.algorithm.n_steps_measurement_param, -1)
# result shape n_samples x n_tuning_configs x n_steps x num_outputs (1 or 2)

In [ ]:
import matplotlib.pyplot as plt
from emitopt.analysis import compute_emit_bmag, compute_emit_bayesian

for i in range(40):
    plt.plot(x[i,0,:,0],bss[i,0,:,0],'C0')
    emit = compute_emit_bmag(
        scale_factor *x[i,0,:,0],bss[i,0,:,0], q_len, rmat_x,thick=False
    )[0]
    print(emit)

plt.ylim(-0.75,14)
plt.xlim(-0.8,0.6)